# SSC JSON GEN

In [ ]:
# find hls tiles given a point

def find_hls_tiles(point=False, band=False, limit=False, collections = ['HLSL30.v2.0', 'HLSS30.v2.0'], date_range = False):

    STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'


    catalog = Client.open(f'{STAC_URL}/LPCLOUD/')



    try:
        x, y = point[0], point[1]
        # print(x,y)
    except TypeError:
        print("Point must be in the form of [lon,lat]")
        raise

    point = geopandas.points_from_xy([x],[y])
    point = point[0]



    # JOHN - THIS IS WHERE YOU WOULD ADD IN SEARCH PARAMETERS
    if date_range:

        search = catalog.search(
            collections=collections, intersects = point, datetime=date_range)
    else:
        search = catalog.search(
            collections=collections, intersects = point)



    # print(f'{search.matched()} Tiles Found...')


    item_collection = search.get_all_items()

    if limit:
        item_collection = item_collection[:limit]

    if band:
        links = []
        if type(band) == list:
            for i in item_collection:
                for b in band:
                    link = i.assets[b].href
                    # print(link)
                    links.append(link)
        
        else:
            for i in item_collection:
                link = i.assets[band].href
                links.append(link)
    
    else:
        links =[]
        for i in item_collection:
            # print(i.assets)
            for key in i.assets:
                if key.startswith('B'):
                    # link = i.assets[key].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')
                    link = i.assets[key].href

                    # print(link)
                    links.append(link)

    return links

def datestdtojd (stddate):
    fmt='%Y-%m-%d'
    sdtdate = datetime.datetime.strptime(stddate, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday
    return(stddate[:4]+str(jdate))

def jdtodatestd (jdate):
    fmt = '%Y%j'
    datestd = datetime.datetime.strptime(jdate, fmt).date()
    out = datestd.strftime( '%Y-%m-%d')
    return out


def ssc_json_gen(aquasat_path, use_restart = True):

    # read in aquasat
    df = pd.read_csv(aquasat_path)

    f = IntProgress(min=0, max=len(df)) # instantiate the bar
    display(f) # display the bar

    fails = 0

    if use_restart:
        # needs to be more robust restarting
        the_range = range(len(glob.glob('./ssc_jsons/*'))+1,len(df))
    else:
        the_range = range(len(df))

    f.value += min(the_range)
    
    for i in the_range:
        lat, lon, date = str(df.iloc[i]['lat']), str(df.iloc[i]['long']), df.iloc[0]['date']
        try:
            # date = datestdtojd('2013-01-13')
            # print(date)
            links = find_hls_tiles(point=[lon, lat])
            if len(links)>0:
                date_list = []
                for l in links:
                    date = l.split('/')[-1].split('.')[3][:7]
                    date_list.append(jdtodatestd(date))

                # date_list = [jdtodatestd(x) for x in date_list]

                # date_list.sort(key=lambda date: datetime.datetime.strptime(date, '%Y-%m-%d'))


                ssc_dict = {}

                date_dict = {}
                for z in list(set(date_list)):
                    

                    date_list = np.asarray(date_list)
                    
                    bool_arr = date_list==z

                    link_list_for_date = np.asarray(links)[bool_arr]

                    link_list_for_date = sorted(link_list_for_date, key = lambda x: x.split('.')[-2][1:])
                    date_dict[z] = link_list_for_date

                ssc_dict[str(lon) +','+ str(lat)]  = date_dict


                # Serializing json
                json_object = json.dumps(ssc_dict, indent=4)
                
                # Writing to sample.json
                with open(f"./ssc_jsons/date_dict_{i}.json", "w") as outfile:
                    outfile.write(json_object)
                
        except Exception as e:
            print(e)
            print('fail')
            fails +=1
        f.value += 1 # signal to increment the progress bar
        time.sleep(.1)
    print(f'Total fails: {fails} ')
    print(f'Total Success: {len(df)-fails}')
    return ssc_dict

In [ ]:
# https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html

# https://rafatieppo.github.io/post/2018_12_01_juliandate/

from pystac_client import Client  
from collections import defaultdict    
import json
import geopandas
from cartopy import crs
import geopandas as gpd
from ipywidgets import IntProgress
from IPython.display import display
import time
import pandas as pd
import glob
import netCDF4
import os
import numpy as np

import datetime
import pandas as pd
import glob
import asyncio

aqua_path = '/data/data/downloads/Aqusat_TSS_v1.csv'

# jj = ssc_json_gen(aqua_path)
    


# multiprocessing


In [ ]:
# find hls tiles given a point

def find_hls_tiles(point=False, band=False, limit=False, collections = ['HLSL30.v2.0', 'HLSS30.v2.0'], date_range = False):

    STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'


    catalog = Client.open(f'{STAC_URL}/LPCLOUD/')



    try:
        x, y = point[0], point[1]
        # print(x,y)
    except TypeError:
        print("Point must be in the form of [lon,lat]")
        raise

    point = geopandas.points_from_xy([x],[y])
    point = point[0]



    # JOHN - THIS IS WHERE YOU WOULD ADD IN SEARCH PARAMETERS
    if date_range:

        search = catalog.search(
            collections=collections, intersects = point, datetime=date_range)
    else:
        search = catalog.search(
            collections=collections, intersects = point)



    # print(f'{search.matched()} Tiles Found...')


    item_collection = search.get_all_items()

    if limit:
        item_collection = item_collection[:limit]

    if band:
        links = []
        if type(band) == list:
            for i in item_collection:
                for b in band:
                    link = i.assets[b].href
                    # print(link)
                    links.append(link)
        
        else:
            for i in item_collection:
                link = i.assets[band].href
                links.append(link)
    
    else:
        links =[]
        for i in item_collection:
            # print(i.assets)
            for key in i.assets:
                if key.startswith('B'):
                    # link = i.assets[key].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/', 's3://')
                    link = i.assets[key].href

                    # print(link)
                    links.append(link)

    return links

def datestdtojd (stddate):
    fmt='%Y-%m-%d'
    sdtdate = datetime.datetime.strptime(stddate, fmt)
    sdtdate = sdtdate.timetuple()
    jdate = sdtdate.tm_yday
    return(stddate[:4]+str(jdate))

def jdtodatestd (jdate):
    fmt = '%Y%j'
    datestd = datetime.datetime.strptime(jdate, fmt).date()
    out = datestd.strftime( '%Y-%m-%d')
    return out



async def find_tiles_and_write(df, i):

    lat, lon, date = str(df.iloc[i]['lat']), str(df.iloc[i]['long']), df.iloc[0]['date']
    try:
        links = find_hls_tiles(point=[lon, lat])
        if len(links)>0:
            date_list = []
            for l in links:
                date = l.split('/')[-1].split('.')[3][:7]
                date_list.append(jdtodatestd(date))
            ssc_dict = {}
            date_dict = {}
            for z in list(set(date_list)):

                date_list = np.asarray(date_list)
                
                bool_arr = date_list==z

                link_list_for_date = np.asarray(links)[bool_arr]

                link_list_for_date = sorted(link_list_for_date, key = lambda x: x.split('.')[-2][1:])
                date_dict[z] = link_list_for_date

            ssc_dict[str(lon) +','+ str(lat)]  = date_dict
            # Serializing json
            json_object = json.dumps(ssc_dict, indent=4)
            
            # Writing to sample.json
            with open(f"./ssc_jsons/date_dict_{i}.json", "w") as outfile:
                outfile.write(json_object)

    except Exception as e:
        print(e)
        print('fail')
        fails +=1



# turn this function async


async def driver(df, the_range):
    await asyncio.gather(*(find_tiles_and_write(i) for i in the_range))

def ssc_json_gen(aquasat_path, use_restart = True):

    # read in aquasat
    df = pd.read_csv(aquasat_path)

    fails = 0

    if use_restart:
        # needs to be more robust restarting
        the_range = range(len(glob.glob('./ssc_jsons/*'))+1,len(df))
    else:
        the_range = range(len(df))
    

    # make new function here that we want to parallelize
    # for i in the_range:
    #     find_tiles_and_write(df, i)

    asyncio.run(driver(df, the_range=the_range))
                





    print(f'Total fails: {fails} ')
    print(f'Total Success: {len(df)-fails}')

In [ ]:
# https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/02_Data_Discovery_CMR-STAC_API.html

# https://rafatieppo.github.io/post/2018_12_01_juliandate/

from pystac_client import Client  
from collections import defaultdict    
import json
import geopandas
from cartopy import crs
import geopandas as gpd
from ipywidgets import IntProgress
from IPython.display import display
import time
import pandas as pd
import glob
import netCDF4
import os
import numpy as np

import datetime
import pandas as pd
import glob
import asyncio

aqua_path = '/data/data/downloads/Aqusat_TSS_v1.csv'

jj = ssc_json_gen(aqua_path)